In [73]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.load import dumps, loads
from operator import itemgetter
from langchain_openai import ChatOpenAI
#from langchain_core.runnables import RunnablePassthrough


In [76]:
pdf_reader="C:/Users/ksais/Downloads/Documents/handbook.pdf"

In [77]:
from PyPDF2 import PdfReader
pdf_reader=PdfReader(pdf_reader)
text = ""
for page in pdf_reader.pages:
    text += page.extract_text()  # Extract text from each page and concatenate

In [78]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema.document import Document

def get_text_chunks_langchain(text):
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    docs = [Document(page_content=x) for x in text_splitter.split_text(text)]
    return docs

def main():
    #a = "Your text from the variable goes here"
    docs = get_text_chunks_langchain(text)
    print(docs)

if __name__ == '__main__':
    main()


[Document(page_content='Zania, Inc.\nZania Employee Handbook\nSeptember 07, 2023TABLE OF CONTENTS\nCORE POLICIES\n4\n1.0 WELCOME\n4\n1.1 A Welcome Policy\n4\n1.2 At-Will Employment\n4\n2.0 INTRODUCTORY LANGUAGE AND POLICIES\n5\n2.1 About the Company\n5\n2.2 Company Facilities\n5\n2.3 Ethics Code\n5\n2.4 Mission Statement\n5\n2.5 Our Organization\n5\n2.6 Revisions to Handbook\n5\n3.0 HIRING AND ORIENTATION POLICIES\n5\n3.1 Accommodations for Pregnant Employees\n5\n3.2 Conflicts of Interest\n6\n3.3 Employment Authorization Verification\n6\n3.4 Employment of Relatives and Friends\n6\n3.5 Job Descriptions\n7\n3.6 New Hires and Introductory Periods\n7\n3.7 Training Program\n7\n4.0 WAGE AND HOUR POLICIES\n7\n4.1 Attendance\n7\n4.2 Business Expenses\n7\n4.3 Direct Deposit\n8\n4.4 Employment Classifications\n8\n4.5 Introduction to Wage and Hour Policies\n8\n4.6 Job Abandonment\n9\n4.7 Paycheck Deductions\n9\n4.8 Recording Time\n9\n4.9 Travel Expenses\n10\n4.10 Use of Employer Credit Cards\n11\

In [79]:
len(docs)

3

In [42]:
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200
K = 3  

In [45]:
file_path="C:/Users/ksais/Downloads/Documents/handbook.pdf"

In [46]:
def create_vector_database(pdf_path):
    loader = PyPDFLoader(pdf_path)
    pages = loader.load_and_split()
    # Split
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=CHUNK_SIZE, 
        chunk_overlap=CHUNK_OVERLAP)
    # Make splits
    splits = text_splitter.split_documents(pages)
    vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())
    retriever = vectorstore.as_retriever()
    return retriever

In [47]:
retriever = create_vector_database(file_path)

In [48]:
print(retriever)

tags=['Chroma', 'OpenAIEmbeddings'] vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000002088E270590>


### Prompt

In [68]:
# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

#generate_queries = (prompt_perspectives | ChatOpenAI(temperature=0) | StrOutputParser() | (lambda x: x.split("\n")))

In [69]:
def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

In [70]:
def retrieval_chain(pdf_path,template,prompt_perspectives):
    retriever = create_vector_database(file_path)
    prompt_perspectives = prompt_perspectives
    generate_queries = (prompt_perspectives | ChatOpenAI(temperature=0) | StrOutputParser() | (lambda x: x.split("\n")))
    retrieval_chain = generate_queries | retriever.map() | get_unique_union
    return retrieval_chain

In [71]:

# Retrieve
question = "Who is the CEO of the company?"
#retrieval_chain = generate_queries | retriever.map() | get_unique_union
retrieval_chain=retrieval_chain(file_path,template,prompt_perspectives)
docs = retrieval_chain.invoke({"question":question})
len(docs)

3

In [72]:
docs

[Document(page_content='Closing Statement\nThank you for reading our handbook. We hope it has provided you with an understanding of our mission, history, and\nstructure as well as our current policies and guidelines. We look forward to working with you to create a successful\nCompany and a safe, productive, and pleasant workplace.\nShruti Gupta, CEO\nZania, Inc.\n45', metadata={'page': 44, 'source': 'C:/Users/ksais/Downloads/Documents/handbook.pdf'}),
 Document(page_content='opportunities for improvement; and to assist the Company in developing effective recruitment and retention strategies. Your\ncooperation in the exit interview process is appreciated.\n5.4 \nOpen Door/Conflict Resolution Process\nZania, Inc. strives to provide a comfortable, productive, legal, and ethical work environment. To this end, we want you to\nbring any problems, concerns, or grievances you have about the work place to the attention of your Manager and, if\nnecessary, to Human Resources or upper level manage

In [74]:
def question_answer(question):
    # RAG
    template = """Answer the following question based on this context:
    {context}
    
    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(template)
    llm = ChatOpenAI(temperature=0)

    final_rag_chain = (
        {"context": retrieval_chain, 
         "question": itemgetter("question")}| prompt| llm| StrOutputParser())
    return final_rag_chain.invoke({"question":question})

In [75]:
print(question_answer("Who is the CEO of the company"))

Shruti Gupta is the CEO of the company Zania, Inc.
